## Function:

- Function 在 R 裡是一個 <font color='red'> object </font>。
- Function 在 R 理是 <font color='red'> first class objects </font>:
    - Function 可以被當成 arguments，傳入另一個 function。
    - 可以有 <font color='red'> nested function </font>。也就是 function 裡面還有 function。
- Function 的 return value 是 function body 內的 <font color='red'> 最後一個 expression</font>。

### Fuction arguments:

- formal arguments: 在 define function 時設的 arguments。
- 用 ```formals(function 名稱)```可以列出 function 有哪些 formal arguments。
- 不是每個 argument 都會用到。
- 在傳入 arguments 時，可以只傳入 argument 名稱的前幾個字(但要確保 R 找的到)。

### Lazy Evaluation:

R 只在需要 function 的 argument 時，才會去找那個 argument 是否存在。
舉例:  
下面我呼叫 f() 時，只傳入 45，所以 ```a = 45``` 而 b 沒有給定。  
程式在執行 ```print(a)``` 時沒有問題，因為他找的到 argument ```a```。  
但在 ```print(b)``` 就會發生錯誤。

In [1]:
f <- function(a, b) {
 print(a)
 print(b)
}
f(45)

[1] 45


ERROR: Error in print(b): 缺少引數 "b"，也沒有預設值


## The "..." argument:

在傳入 function 的 argument 時，如果想從其他 function 拿取 argument，但又不想重新寫一遍，可以用 ```...```。  
一個應用的地方是 generic functions。

In [3]:
myplot <- function(x, y, type = "l", ...) {
 plot(x, y, type = type, ...)
}

```...```在不知道 function 有多少 input 時也會出現，比如:

In [5]:
args(paste)

function (..., sep = " ", collapse = NULL, recycle0 = FALSE) 
NULL

```paste()```這個 function 將 object 合起來，但 ```paste()``` 一開始不會知道使用者會傳入多少 object，故要用```...```。

要注意的是，使用```...```後，後面的 arguments 必須 <font color='red'> 照順序且不能用縮寫 </font>，舉例:

In [6]:
args(paste)

function (..., sep = " ", collapse = NULL, recycle0 = FALSE) 
NULL

因此，```sep``` 必須照順序，且不能用縮寫:

In [7]:
paste("a", "b", se=":")

[1] "a b :"

可以看到，用縮寫的結果就是 R 直接無視你 argument 的名稱，並把```:```當作另一個要被 paste 在一起的 object。

In [8]:
paste("a", "b", sep=":")

[1] "a:b"

這裡感覺需要查詢更多資訊。

[筆記來源](https://d3c33hcgiwev3.cloudfront.net/_a06bfc4457974762032b75163ca1b7b0_scoping_binding.pdf?Expires=1602201600&Signature=kPOuLasIzcqiTCWn9ND8CTvIVz6zetmI6Whfg18V5DPMMHQWZ6yA9PkZ9gJjYZLuyO-EQPs~rlMkE0maMHtK8f6OvRZ-~ISgYt1pYI-EIb4THXWCi3nW5uAip4f8mcFLgRNRachAG9CXK18CZAC9K08euHIL1H2WWtbkjvf0LoI_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A)

在 R 裡，一個 ```symbol```會搭配一個 ```value```。

```R
lm <- function(x){x * X}
```
我們創了一個 function，叫做 ```lm```。 
那 R 要如何知道 ```lm``` 這個 ```symbol```要搭配哪一個 value?  
因為其實 R 裡面的 ```stats``` package 就有一個 ```lm```。    
1. 他會先去 ```global environment```(user's workspace)找看看有沒有對應的 symbol name。
2. 找不到的話再去 ```search list``` 的 ```packages``` 裡的 ```namespace```([命名空間](https://zh.wikipedia.org/wiki/%E5%91%BD%E5%90%8D%E7%A9%BA%E9%97%B4)) 找。(用 ```search()```)

In [9]:
search()

[1] ".GlobalEnv"        "jupyter:irkernel"  "package:stats"    
 [4] "package:graphics"  "package:grDevices" "package:utils"    
 [7] "package:datasets"  "package:methods"   "Autoloads"        
[10] "package:base"

### Binding values to symbol:
1. User's workspace 的 global environment 會在 search list 的第一位。(base package 一定在最後)。
2. Search list 上的 <font color='red'> packages </font> 的順序是有意義的。
3. 如果使用者載入了 package with ```library```。那那個```library```的 namespace 就會被放到順位 2。(其他下移)
4. Functions 跟 non-function 有分離的 ```namespace```。(也就是有可能有一個 object 叫 c，同時另一個 function 也叫 c)

## R 的 <font color='red'> Scoping Rule </font>:

Scopoing rule 功能: 決定一個 function 裡的 <font color='red'> free variable </font> 會被分配到哪一個值。    
R 使用: *lexical scoping(static scoping)* (相對於 dynamic scoping)。   
說到底，就是 R 怎麼用 search list 來將 symbol 跟 value 配對(bind a value to a symbol)。   
Lexical scoping 對於統計計算很有用。

舉例:  
```R
f <- function(x, y) {
 x^2 + y / z
}
```
---
```f``` function 裡有一個 ```z``` variable。  
他既不是 formal argument 也不是 local variable。

> Lexical scoping in R means that
the values of free variables are searched for <font color='red'>in the environment</font> <font color='green'>in which the function was defined</font>.

那甚麼是 envrionment?
---
- A collection of (symbol, value) pairs.  
- 每個 environment 都有一個 *parent environment*。(唯一一個沒有 parent 的 environment 是 <font color='green'> empty environment</font>)
- 一個 envrionment 可能有很多個 *children*。
- <font color='green'> 一個 function + 一個 environment </font> 稱為 <font color='red'>*closure*(或 *function closure*)</font>。
